# 1. IMPORTING NECESSARY LIBRARIES:

In [3]:
import pandas as pd 
import numpy as np
import base64
import seaborn as sns
import matplotlib.pyplot as plt
import os
import random
import gc
import optuna

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from mlxtend.preprocessing import minmax_scaling
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
from scipy import stats
from scipy.stats import skew
from scipy.signal import argrelmin
from sklearn.ensemble import(RandomForestClassifier, RandomForestRegressor, AdaBoostClassifier, AdaBoostRegressor,
                            GradientBoostingClassifier, GradientBoostingRegressor,
                            ExtraTreesClassifier, ExtraTreesRegressor, VotingClassifier)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn import datasets, linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans

pd.set_option('display.max_columns', None)

import eli5
from eli5.sklearn import PermutationImportance
import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,BatchNormalization,Dropout
from tensorflow.keras import utils
import tensorflow_addons as tfa
import keras_tuner
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
from kerastuner import HyperParameters, Objective

import warnings

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\2211592\AppData\Local\Temp\1\ipykernel_13884\3242579606.py:70: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization


# 2. LOADING DATASETS AND SETTING UP TOOLS:

In [4]:
#loading the Test and the Train dataset

test = pd.read_csv('test.csv')
test.drop('id', axis = 1, inplace = True)
test['adv_val'] = 0

train = pd.read_csv('train.csv')
train.drop('id', axis = 1, inplace = True)
train['adv_val'] = 1

original = pd.read_csv('ParisHousing.csv')
original['adv_val'] =  2

sample = pd.read_csv('sample.csv')

In [5]:
print("These are the first 5 row of our dataset")
original.head()

These are the first 5 row of our dataset


,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price,adv_val
0,75523,3,0,1,63,9373,3,8,2005,0,1,4313,9005,956,0,7,7559081.5,2
1,80771,39,1,1,98,39381,8,6,2015,1,0,3653,2436,128,1,2,8085989.5,2
2,55712,58,0,1,19,34457,6,8,2021,0,0,2937,8852,135,1,9,5574642.1,2
3,32316,47,0,0,6,27939,10,4,2012,0,1,659,7141,359,0,3,3232561.2,2
4,70429,19,1,1,90,38045,3,7,1990,1,0,8435,2429,292,1,4,7055052.0,2


In [6]:
original.shape

(10000, 18)

In [12]:
original.columns

Index(['squareMeters', 'numberOfRooms', 'hasYard', 'hasPool', 'floors',
       'cityCode', 'cityPartRange', 'numPrevOwners', 'made', 'isNewBuilt',
       'hasStormProtector', 'basement', 'attic', 'garage', 'hasStorageRoom',
       'hasGuestRoom', 'price', 'adv_val'],
      dtype='object')

In [7]:
sample.shape

(15154, 2)

In [8]:
train.shape

(22730, 18)

In [9]:
test.shape

(15154, 17)

In [10]:
sets={'train':train,'test':test,'original':original}

In [11]:
sets

{'train':        squareMeters  numberOfRooms  hasYard  hasPool  floors  cityCode  \
 0             34291             24        1        0      47     35693   
 1             95145             60        0        1      60     34773   
 2             92661             45        1        1      62     45457   
 3             97184             99        0        0      59     15113   
 4             61752            100        0        0      57     64245   
 ...             ...            ...      ...      ...     ...       ...   
 22725         55825             84        1        0      70     12031   
 22726         65870             88        1        0      49     23197   
 22727         93192             42        1        0      39      8539   
 22728         65797             86        1        0      89     23197   
 22729         82244             18        1        0      38     86728   
 
        cityPartRange  numPrevOwners  made  isNewBuilt  hasStormProtector  \
 0          

In [9]:
def dropping_duplicates(data):
    data.drop_duplicates(inplace = True)
    
for n in {'train':train,'original':original}:
    print(f"A number of duplicated rows in {n} is {sets[n].duplicated().sum()}, they were dropped")
    dropping_duplicates(sets[n])

A number of duplicated rows in train is 0, they were dropped
A number of duplicated rows in original is 0, they were dropped


### General Settings

In [10]:
gpu_switch = 'ON'

#RepeatedStratifiedKFold

n_splits = 5
n_repeats =10
sm=SMOTE(sampling_strategy='minority')
#cv = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats = n_repeats, random_state=2023)
cv = RepeatedKFold(n_splits=n_splits, n_repeats = n_repeats, random_state=2023)
#cv = GroupKFold(n_splits=n_splits)


#Define Weights
weights = {0: 0.5009553158705701, 1: 262.19354838709677}

### Creating functions for step-by-step analyzing:

### Function 1:

In [11]:
def LGBM_get_score(X,y):
    param = {'metric': 'rmse', 
            'early_stopping_round': 300, 
            'n_estimators': 5000}
    results=[]
    for i,(train_index, test_index) in enumerate(cv.split(X, y)):
        train_X, valid_X = X.iloc[train_index], X.iloc[test_index]
        train_y, valid_y = y.iloc[train_index], y.iloc[test_index]
        model = lgb.LGBMRegressor(**param).fit(train_X,train_y,
                                            eval_set=[(valid_X,valid_y)],
                                            callbacks=[lgb.log_evaluation(period=0, show_stdv=False)]) 
        result=np.sqrt(mean_squared_error(valid_y,model.predict(valid_X))) 
        results.append(result)
        print (f'\033[0;33;40m Step#{i} Auc result = {result} \033[0;30;0m')
    print (f'\033[0;35;40m Final LGBM Result = {sum(results)/len(results)} \033[0;30;0m')

### Function 2: 

In [12]:
def RF_get_score(X,y):
    param = {'n_jobs':-1}
    results=[]
    for i,(train_index, test_index) in enumerate(cv.split(X, y)):
        train_X, valid_X = X.iloc[train_index], X.iloc[test_index]
        train_y, valid_y = y.iloc[train_index], y.iloc[test_index]
        model = RandomForestRegressor(**param).fit(train_X,train_y) 
        result=np.sqrt(mean_squared_error(valid_y,model.predict(valid_X))) 
        results.append(result)
        print (f'\033[0;33;40m Step#{i} Auc result = {result} \033[0;30;0m')
    print (f'\033[0;35;40m Final LGBM Result = {sum(results)/len(results)} \033[0;30;0m')

In [13]:
#########################################################################
### The function for data distribution analysis between two datasets  ###
#########################################################################

def viz_comp (data1,data2,title):
    n_bins = 50
    histplot_hyperparams = {
        'kde':True,
        'alpha':0.4,
        'stat':'percent',
        'bins':n_bins
    }
    cols=num_cols
    fig, ax = plt.subplots(len(cols)//3+1,3, figsize=(20, 20))
    ax = ax.flatten()

    for i, column in enumerate(cols):
        sns.histplot(
            data1[column], label='Train',
            ax=ax[i], color='green', **histplot_hyperparams)

    for i, column in enumerate(cols):
        sns.histplot(
            data2[column], label='Train',
            ax=ax[i], color='red', **histplot_hyperparams)
    ax[0].set_title(title, fontstyle='normal',size=25)

In [14]:
##############################
### Permutation importance ###
##############################

def permutation_importance(model,X,y):
    permute = PermutationImportance(model,random_state=2023,n_iter =10,cv=5).fit(X, y)
    eli5.show_weights(permute, feature_names = X.columns.tolist(),top=50)
    values = dict(zip(list(train.columns),list(permute.feature_importances_)))
    sorted_dict = {}
    sorted_keys = sorted(values, key=values.get)
    for w in sorted_keys:
        sorted_dict[w] = values[w]

# 3. MISSING INFORMATION:

In [15]:
def miss_values_check(data,n):
    print(f'\033[0;33;40m A number of NaN values in {n} is {data.isnull().sum().sum()} \033[0;30;0m')
    if data.isnull().sum().sum() >0:
        sns.heatmap(data.isnull())
for n in sets:
    miss_values_check(sets[n],n)

 A number of NaN values in train is 0 
 A number of NaN values in test is 0 
 A number of NaN values in original is 0 


# 4. DATA PREPROCESSING:

## 4.1 ENCODING AND IMPUTING

In [ ]:
performing hyper parameter tuning using Optuna withXGBoost as the model:

In [ ]:
Now the best set of hyperparameters obtained from Optuna optimization process for the XGBoost Model

In [ ]:
Now using the best hyperparameters we are evaluating its performance using cv